In [6]:
import os 
#open_api_key="private key"
os.environ["OPENAI_API_KEY"]=open_api_key

In [7]:
## Basic Prompt Summarization
from langchain.chat_models import ChatOpenAI
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
import ast
import pandas as pd
import matplotlib.pyplot as plt
import re


In [8]:
def parseTextAndCreateAllRisks(ticker):
    llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')
    
    text = ""
    
    chat_messages=[
        SystemMessage(content='You are an expert assistant with expertize in summarizing speeches'),
        HumanMessage(content=f'Please provide the top risk categories and the number of risks for the given year:\n TEXT: {text}')
    ]
    
    final_output = ""
    all_results = []

    # Set a base path to the data directory
    base_path = os.path.join('..', 'data')
    item1a_path = os.path.join(base_path, 'item1a_filings')
    directory_path = os.path.join(item1a_path, ticker)
    
    for filename in os.listdir(directory_path):
        #print(filename)
        if filename.endswith("_item1a.txt"):  # Filter for specific file types
            file_path = os.path.join(directory_path, filename)
            with open(file_path, "r", encoding='utf-8') as file:
                text = file.read()
    
            # Extract the year from the filename
            year = filename.split("_")[1]
    
            # Splitting the text into manageable chunks
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=200)
            chunks = text_splitter.create_documents([text])
    
            chunks_prompt="""
                            for the given year summarize the risk factors {text} in less than 10000 words.
                            Summary:
                            """
            map_prompt_template=PromptTemplate(input_variables=['text'], template=chunks_prompt)
    
            final_combine_prompt="""
                                    Given the following text, Summarize the top 20 risk factors and include the year as a tuple for each risk for the given year, 
                                    tuples of year as number, risk as a string in quotes for the same year,
                                    {text}
                                """
    
            final_combine_prompt_template=PromptTemplate(input_variables=['text'], template=final_combine_prompt)
            
            # Define the summarization chain
            summary_chain = load_summarize_chain(
                llm=llm,
                chain_type='map_reduce',
                map_prompt=map_prompt_template,
                combine_prompt=final_combine_prompt_template,
                verbose=False
            )
    
            # Run the summarization process
            output_summary = summary_chain.run(chunks)
            #print("\n\n\nOutput_Summary")
            #print(output_summary)
            final_output += output_summary
    
            actual_data = ast.literal_eval(output_summary)
            all_results.extend(actual_data)
    
    #print("all_results ...............")
    all_results

    df = pd.DataFrame(all_results, columns=['Year', 'Risk'])
    # Remove quotes from the 'Risk' column
    df['Risk'] = df['Risk'].str.replace('"', '')
    
    # Add quotes around the 'Risk' column
    df['Risk'] = '"' + df['Risk'] + '"'

    output_directory_path = os.path.join(output_directory_path, "cleaned_tuples")
    # Construct the output file path
    output_file = os.path.join(directory_path, f'{ticker}_cleaned_tuples.csv')
    
    # Write the DataFrame to CSV with modified 'Risk' column
    df.to_csv(output_file, columns=['Year', 'Risk'], index=False)

In [5]:
parseTextAndCreateAllRisks("AAPL") #Invoke the code.

C:\Users\rajun\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
C:\Users\rajun\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}